# Black Scholes Exercise 7: Python Parallelism - Apply
Use python's pool-apply concept to execute embarrasingly parallel computation

In [ ]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

In [ ]:
from numpy import log, invsqrt, exp, erf
# Black Scholes kernel
def black_scholes(nopt, price, strike, t, rate, vol):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * erf(w1)
    d2 = 0.5 + 0.5 * erf(w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return (call, put)

### Define a wrapper
   - partition the arrays
   - apply to a pool of processes or threads
   - get results

In [ ]:
from multiprocessing import cpu_count

def black_scholes_apply(pool, nopt, price, strike, t, rate, vol):
    noptpp = int(nopt/cpu_count())
    call = np.empty(nopt, dtype=np.float64)
    put = np.empty(nopt, dtype=np.float64)
    asyncs = [pool.apply_async(black_scholes, (noptpp, price[i:i+noptpp], strike[i:i+noptpp], t[i:i+noptpp], rate, vol)) for i in range(0, nopt, noptpp)]
    for a,i in zip(asyncs, range(len(asyncs))):
        call[i:i+noptpp], put[i:i+noptpp] = a.get()
    return call, put

### Instantiate a pool

In [ ]:
import multiprocessing.pool
pool = multiprocessing.pool.ThreadPool(cpu_count())

In [ ]:
# Now run and timeit

## Play with number of CPUs
## Use other pools, like TBB's pool or multiprocessing

In [ ]:
import ...
pool = ...
%timeit ...